<p style="font-weight:bold;font-size:200%">Events and Insider Tradings</p>

- Author: Yichen Luo
- Date: 2022-09-01

In [1]:
import time
import random
import numpy as np
import pandas as pd
from datetime import timezone
import datetime
import tweepy
import requests
import configparser
import matplotlib.pyplot as plt
import re
import os
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
RD = '/home/team_defi/02_rdata/'
WD = '/home/team_defi/03_wdata/'
RES = '/home/team_defi/04_result/'

# Root List

In [153]:
# df = pd.read_csv(f'{WD}root_new/list_lp/panel_ret_buy.csv')
# for idx, row in df.drop_duplicates('contractAddress').iterrows():
#     index = row['id']
#     contractAddress = row['contractAddress']
    
#     if row['contractAddress'] == row['token0_id']:
#         df.loc[df['contractAddress'] == contractAddress, 'insider_token_name'] = row['token0_name']
#     else:
#         df.loc[df['contractAddress'] == contractAddress, 'insider_token_name'] = row['token1_name']

# lst = df.drop_duplicates(['contractAddress','insider_token_name','chain'])[['contractAddress','insider_token_name', 'chain']]
# lst['twitter'] = -1
# lst['transaction'] = -1
# lst.to_csv(f'{WD}tweet_new/list/token_list.csv', index=False)

# Event: Root

In [161]:
api = tweepy_config()
lst = pd.read_csv(f'{WD}tweet_new/list/token_list.csv')
done = lst.loc[lst['twitter'] != -1].shape[0]
total = lst.shape[0]

for idx, row in lst.loc[lst['twitter'] == -1].iterrows():
    try:
        done += 1
        contractAddress = row['contractAddress']
        insider_token_name = row['insider_token_name']
        chain = row['chain']

        name = official_twitter(contractAddress=contractAddress,
                                chain=chain,
                                tokenName = insider_token_name)

        users = api.search_users(name)
        
        if len(users) == 0: 
            lst.loc[lst['insider_token_name'] == insider_token_name, 'transaction'] = 0
            lst.to_csv(f'{WD}tweet_new/list/token_list.csv', index=False)
            print(f'{insider_token_name} no users info.')
            continue
            
        dft = get_tweets(user_id=users[0].id)
        dft.to_csv(f'{WD}tweet_new/csv/{contractAddress}_tweets.csv', index=False)
        
        dft['text'] = dft['text'].apply(lambda x: x.lower())
        dft = dft.loc[[True if re.search('listing', i) else False for i in dft['text']]]
        dft['timestamp'] = pd.to_datetime(dft['timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
        events = dft['timestamp'].unique()

        final = []
        for event in events:
            txn = []
            for idx, row in df.loc[df['contractAddress'] == contractAddress].iterrows():
                index = row['id']
                if (pd.to_datetime(row['timestamp']) >= (pd.to_datetime(event) - datetime.timedelta(days=3)))&(pd.to_datetime(row['timestamp']) <= (pd.to_datetime(event) + datetime.timedelta(days=3))):
                    txn.append(df.loc[df['id'] == index])

            if len(txn) != 0:
                txn = pd.concat(txn)
                txn['event'] = event
                txn['time_delta'] = pd.to_datetime(txn['timestamp']) - pd.to_datetime(txn['event'])
                final.append(txn)


        length = len(final)
        if length!= 0: 
            final = pd.concat(final)
            final.to_csv(f'{WD}tweet_new/csv/{contractAddress}_txns.csv', index=False)
            lst.loc[lst['insider_token_name'] == insider_token_name, 'transaction'] = length
        else: 
            lst.loc[lst['insider_token_name'] == insider_token_name, 'transaction'] = 0
        
        
        if name == insider_token_name: 
            lst.loc[lst['insider_token_name'] == insider_token_name, 'twitter'] = 2
        else:
            lst.loc[lst['insider_token_name'] == insider_token_name, 'twitter'] = 1
            
        print(f'{insider_token_name} have {length} event-related transactions. {done}/{total}')
        lst.to_csv(f'{WD}tweet_new/list/token_list.csv', index=False)
    except Exception as e:
        print(f'{insider_token_name} Error: {e}')

The Show Goes On no users info.
t.me/skin_port no users info.
RunwayDiamond no users info.
NMR Risk Management Technology no users info.
BioBit no users info.
Worldcoin.org  no users info.
SPIKE INU no users info.
Bill Gate Token no users info.
FeeToken no users info.
THRAX Error: 401 Unauthorized
Not authorized.
Shiba Rogan no users info.
Rubble Inu no users info.
RebootETH no users info.
RebootETH no users info.
baby Marshall Rogan Inu no users info.
Heal My Wallets no users info.
Heal My Bags no users info.
Average Joe Crypto no users info.
BUYBACK MAN no users info.
Rainbow Kitty Inu no users info.
GianniInfantinoInu no users info.
ParmaNobi no users info.
Chad Apes Club no users info.
Sakamoto Inu no users info.
Millionaire Chads Club no users info.
Corel Inu no users info.
ShumoNobi no users info.
Meme Coin Degens no users info.
Espresso [IEO] no users info.
Fuck Panic Sellers no users info.
BigBuyChads no users info.
Hollywood Shib no users info.
AutoDonatePoor no users info.
Ru

# Analysis

In [174]:
lst = pd.read_csv(f'{WD}tweet_new/list/token_list.csv')
dfs = []
for idx, row in lst.loc[lst['transaction'] >0].iterrows():
    try:
        contractAddress = row['contractAddress']
        df = pd.read_csv(f'{WD}tweet_new/csv/{contractAddress}_txns.csv')
        dfs.append(df)
    except:
        pass
    
dfs = pd.concat(dfs)
dfs

,Unnamed: 0,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_root,contractAddress,from,to,lp,buy,ret_1,ret_2,ret_3,ret_-1,ret_-2,ret_-3,insider_token_name,event,time_delta
0,30,719.771991,0x0df250704658224ca55ae95e82bae6e63b5aaefa4db47f5c2f137e3c36529057#708,211,0x83da32c690afc12b08621cdae37268eeda4a964c,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,5535243264563745790662704,-191389.0,2021-08-20 12:05:42,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0df250704658224ca55ae95e82bae6e63b5aaefa4db47f5c2f137e3c36529057,0.000000e+00,4.600000e+07,0.224189,0.000000,True,uniswapv3_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,-719.771991,-0.042418,-0.066189,-0.013041,0.139668,-0.070384,-0.345031,Tiger King,2021-08-17 21:21:39,2 days 14:44:03
1,38,7478.974558,0x63dcb02114a962bbb82a1fb50f1f8ea27aaf4637fcbce5f7219753ed8b580e12#83,48,0x83da32c690afc12b08621cdae37268eeda4a964c,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,5753880147597764424946029,-190614.0,2021-08-18 02:42:10,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x63dcb02114a962bbb82a1fb50f1f8ea27aaf4637fcbce5f7219753ed8b580e12,0.000000e+00,4.630000e+08,2.473233,0.000000,True,uniswapv3_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,-7478.974558,0.139668,0.146603,0.104186,-0.274647,NaN,NaN,Tiger King,2021-08-17 21:21:39,0 days 05:20:31
2,57,5565.306641,0xf5ad728326082b1d4befa92a5338c33284b1ffa1916f2b1a10fc3112a1d96cd9#241,6,0x83da32c690afc12b08621cdae37268eeda4a964c,0x83da32c690afc12b08621cdae37268eeda4a964c,0xe592427a0aece92de3edee1f18e0157c05861564,5435112145683137373079717,-191754.0,2021-08-18 12:05:35,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf5ad728326082b1d4befa92a5338c33284b1ffa1916f2b1a10fc3112a1d96cd9,3.968941e+08,0.000000e+00,0.000000,1.850000,True,uniswapv3_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,5565.306641,0.139668,0.146603,0.104186,-0.274647,NaN,NaN,Tiger King,2021-08-17 21:21:39,0 days 14:43:56
3,1182,4707.722246,0x1abac089b4463e0fb0d10d24e720200f73bb7b7e3e2a8f8ab0d934a1fe8fcebe-0,8,NaN,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,NaN,NaN,2021-08-16 15:23:20,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1abac089b4463e0fb0d10d24e720200f73bb7b7e3e2a8f8ab0d934a1fe8fcebe,1.921182e+08,0.000000e+00,0.000000,1.471628,True,uniswapv2_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0x83da32c690afc12b08621cdae37268eeda4a964c,0x83da32c690afc12b08621cdae37268eeda4a964c,False,4707.722246,-0.274647,-0.484699,-0.345031,NaN,NaN,NaN,Tiger King,2021-08-17 21:21:39,-2 days +18:01:41
4,1183,32399.939227,0x4394130f762c7ab53db85c7923d50f1e3c8a89119d5f925bc1f85c36dc1c39b3-0,8,NaN,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,NaN,NaN,2021-08-16 20:16:13,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x4394130f762c7ab53db85c7923d50f1e3c8a89119d5f925bc1f85c36dc1c39b3,1.273797e+09,0.000000e+00,0.000000,10.300000,True,uniswapv2_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0x83da32c690afc12b08621cdae37268eeda4a964c,0x83da32c690afc12b08621cdae37268eeda4a964c,False,32399.939227,-0.274647,-0.484699,-0.345031,NaN,NaN,NaN,Tiger King,2021-08-17 21:21:39,-2 days +22:54:34
...,...,...,...,...,...,...,...,...,...,..

In [176]:
dfs.drop_duplicates('id').shape[0]

1417

## Descriptive Statistics

In [169]:
lst = pd.read_csv(f'{WD}tweet_new/list/token_list.csv')
lst.shape[0]
lst['twitter'].value_counts()
lst.loc[lst['transaction'] >0].shape[0]
lst.loc[(lst['transaction'] >0)&(lst['twitter']==1)].shape[0]

936

 2    529
 1    209
-1    198
Name: twitter, dtype: int64

98

61

## Insider Buy and Sell Tendency

In [175]:
print('buy before')
num1 = dfs.loc[(dfs['buy'] > 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() < 0)].shape[0]
amo1 = dfs.loc[(dfs['buy'] > 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() < 0), 'buy'].sum()
print(f'Aggregate Frenquency: {num1}')
print(f'Aggregate Dollar Amount: {amo1}')
print(f'Average Dollar Amount: {amo1/num1}')
print('\nsell before')
num2 = dfs.loc[(dfs['buy'] < 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() < 0)].shape[0]
amo2 = dfs.loc[(dfs['buy'] < 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() < 0), 'buy'].sum()
print(f'Aggregate Frenquency: {num2}')
print(f'Aggregate Dollar Amount: {amo2}')
print(f'Average Dollar Amount: {amo2/num2}')
print('\nbuy after')
num3 = dfs.loc[(dfs['buy'] > 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() > 0)].shape[0]
amo3 = dfs.loc[(dfs['buy'] > 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() > 0), 'buy'].sum()
print(f'Aggregate Frenquency: {num3}')
print(f'Aggregate Dollar Amount: {amo3}')
print(f'Average Dollar Amount: {amo3/num3}')
print('\nsell after')
num4 = dfs.loc[(dfs['buy'] < 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() > 0)].shape[0]
amo4 = dfs.loc[(dfs['buy'] < 0)&(pd.to_timedelta(dfs['time_delta']).dt.total_seconds() > 0), 'buy'].sum()
print(f'Aggregate Frenquency: {num4}')
print(f'Aggregate Dollar Amount: {amo4}')
print(f'Average Dollar Amount: {amo4/num4}')

dfs.shape[0]

buy before
Aggregate Frenquency: 1428
Aggregate Dollar Amount: 12083157.248403352
Average Dollar Amount: 8461.594711767053

sell before
Aggregate Frenquency: 969
Aggregate Dollar Amount: -10007966.314335292
Average Dollar Amount: -10328.13861128513

buy after
Aggregate Frenquency: 1144
Aggregate Dollar Amount: 7053314.341094473
Average Dollar Amount: 6165.48456389377

sell after
Aggregate Frenquency: 962
Aggregate Dollar Amount: -13391827.29476795
Average Dollar Amount: -13920.818393729678


4503

## Event Time Span

In [ ]:
df = pd.read_csv(f'{WD}root_new/list_lp/panel_ret_buy.csv')
df['date'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%d')
df

contractAddress = '0x44d09156c7b4acf0c64459fbcced7613f5519918'
chain = 'polygon'

dft = pd.read_csv(f'{WD}tweet_new/csv/{contractAddress}_tweets.csv')
dft['date'] = pd.to_datetime(dft['timestamp']).dt.strftime('%Y-%m-%d')
dft.sort_values('date', ascending=True, inplace=True)
start = dft['date'].min()
end = dft['date'].max()

start
end
dft['timestamp'].unique()
dft['date'].unique()

pd.to_datetime(start) + datetime.timedelta(days=3)
# txn = df.loc[(df['contractAddress'] == contractAddress)&
#              (df['chain'] == chain)&
#              (pd.to_datetime(df['date']) > pd.to_datetime(start) + datetime.timedelta(days=3))&
#              (pd.to_datetime(df['date']) < pd.to_datetime(end) - datetime.timedelta(days=3))].copy()

,Unnamed: 0,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_root,contractAddress,from,to,lp,buy,ret_1,ret_2,ret_3,ret_-1,ret_-2,ret_-3,date
0,0,209.294150,0xe91b835550a1a50b26a0646fa3e1def942a78ea5ce7349529a30700eff08ab11#33823,266,0x008295a98dcd428b680bc1db7a743f1d3993b4d4,0x008295a98dcd428b680bc1db7a743f1d3993b4d4,0xe592427a0aece92de3edee1f18e0157c05861564,9400732126353364784919992,-180796.0,2021-11-22 02:43:11,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe91b835550a1a50b26a0646fa3e1def942a78ea5ce7349529a30700eff08ab11,3.541485e+06,0.000000,0.000000,0.05,True,uniswapv3_ethereum,ethereum,0x008295a98dcd428b680bc1db7a743f1d3993b4d4,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,209.294150,0.026956,-0.143178,-0.102227,-0.580864,-0.503014,-0.738577,2021-11-22
1,1,439.630640,0xd1644c62998cbb5fa03b412ab9c62cd560706e04d2399fd86860fcd10159c499#4,100,0x240bf95c3df6fc0206028a073555e251aa67458b,0x240bf95c3df6fc0206028a073555e251aa67458b,0xe592427a0aece92de3edee1f18e0157c05861564,792833476219372735929747753,-92095.0,2021-09-07 22:15:51,TRIBE,Wrapped Ether,0x4ffcad848248d4931374336fcaccf263638413ae,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd1644c62998cbb5fa03b412ab9c62cd560706e04d2399fd86860fcd10159c499,1.294447e+03,0.000000,0.000000,0.13,True,uniswapv3_ethereum,ethereum,0x240bf95c3df6fc0206028a073555e251aa67458b,0x4ffcad848248d4931374336fcaccf263638413ae,NaN,NaN,False,439.630640,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-07
2,2,325.499103,0x099a3083fd9a904646021f522bef9d887e6ab07b5171a59ca278c4edce13c43b#546,282,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0xe592427a0aece92de3edee1f18e0157c05861564,108654985078874190449425007,-131845.0,2021-08-17 22:21:44,INCH TOKEN,USD Coin,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x099a3083fd9a904646021f522bef9d887e6ab07b5171a59ca278c4edce13c43b,0.000000e+00,170035.300000,325.499103,0.00,True,uniswapv3_ethereum,ethereum,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,NaN,NaN,False,-325.499103,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-17
3,3,235.663410,0x2f9ee0617b0024f000eda6cef85507776e2b7d2f2f637858764eb2c652f6c40e#619,133,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0xe592427a0aece92de3edee1f18e0157c05861564,80395462705354980614036351,-137870.0,2021-09-11 23:57:32,INCH TOKEN,USD Coin,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x2f9ee0617b0024f000eda6cef85507776e2b7d2f2f637858764eb2c652f6c40e,0.000000e+00,224977.000000,235.663410,0.00,True,uniswapv3_ethereum,ethereum,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,NaN,NaN,False,-235.663410,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-11
4,4,1067.164913,0x8e7be63d557448574c91c0ff62e26339473a08906565e79a034db46ac67dea95#378,369,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0xe592427a0aece92de3edee1f18e0157c05861564,131061056549767839461752140,-128095.0,2021-08-02 14:29:56,INCH TOKEN,USD Coin,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8e7be63d557448574c91c0ff62e26339473a08906565e79a034db46ac67dea95,0.000000e+00,359218.340216,1067.164913,0.00,True,uniswapv3_ethereum,ethereum,0xd42b6798e0ea3c455aa31a15e8e582439154e5cc,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,NaN,NaN,False,-1067.164913,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15465,5,42.352345,0xe9a64614f7eeef8eb77a7b5e29258293e69e6eaecbbf6df23855e80edc39031a-2,11,NaN,NaN,0x16327e3fbdaca3bcf7e38f5af2599d2ddc33ae52,NaN,NaN,2022-05-12 20:25:29,Wra

'2020-08-21'

'2022-09-01'

array(['2020-08-21 12:58:58+00:00', '2020-08-22 09:27:54+00:00',
       '2020-08-22 14:48:41+00:00', ..., '2022-09-01 07:16:24+00:00',
       '2022-09-01 09:40:19+00:00', '2022-09-01 12:23:49+00:00'],
      dtype=object)

array(['2020-08-21', '2020-08-22', '2020-08-23', '2020-08-24',
       '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28',
       '2020-08-29', '2020-08-30', '2020-08-31', '2020-09-01',
       '2020-09-02', '2020-09-03', '2020-09-04', '2020-09-05',
       '2020-09-06', '2020-09-07', '2020-09-08', '2020-09-09',
       '2020-09-10', '2020-09-11', '2020-09-12', '2020-09-14',
       '2020-09-15', '2020-09-16', '2020-09-18', '2020-09-19',
       '2020-09-20', '2020-09-21', '2020-09-22', '2020-09-24',
       '2020-09-25', '2020-09-27', '2020-09-29', '2020-09-30',
       '2020-10-01', '2020-10-03', '2020-10-04', '2020-10-05',
       '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09',
       '2020-10-10', '2020-10-11', '2020-10-12', '2020-10-13',
       '2020-10-14', '2020-10-15', '2020-10-17', '2020-10-18',
       '2020-10-20', '2020-10-22', '2020-10-23', '2020-10-24',
       '2020-10-25', '2020-10-26', '2020-10-27', '2020-10-28',
       '2020-10-29', '2020-10-30', '2020-10-31', '2020-

Timestamp('2020-08-24 00:00:00')

In [2]:
df = pd.read_csv(f'{WD}root_new/list_lp/panel_ret_buy.csv')
df['date'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%d')

for day in [1, 2, 3]:
    df[f'event_{day}'] = False

dfs = []
for df_idx, df_row in df.drop_duplicates(['contractAddress', 'chain']).iterrows():
    try:
        contractAddress = df_row['contractAddress']
        chain = df_row['chain']
        
        dft = pd.read_csv(f'{WD}tweet_new/csv/{contractAddress}_tweets.csv')
        dft['date'] = pd.to_datetime(dft['timestamp']).dt.strftime('%Y-%m-%d')

        start = dft['date'].min()
        end = dft['date'].max()

        txn = df.loc[(df['contractAddress'] == contractAddress)&
                     (df['chain'] == chain)&
                     (pd.to_datetime(df['date']) > pd.to_datetime(start) + datetime.timedelta(days=3))&
                     (pd.to_datetime(df['date']) < pd.to_datetime(end) - datetime.timedelta(days=3))].copy()

        dft['text'] = dft['text'].apply(lambda x: x.lower())
        dft = dft.loc[[True if re.search('listing', i) else False for i in dft['text']]]
        events = dft['date'].unique()

        for event in events:
            for idx, row in txn.iterrows():
                index = row['id']
                date = row['date']
                for day in [1, 2, 3]:
                    if (pd.to_datetime(date) > pd.to_datetime(event) - datetime.timedelta(days=day))&(pd.to_datetime(date) < pd.to_datetime(event) + datetime.timedelta(days=day)):
                        txn.loc[(txn['id'] == index), f'event_{day}'] = True

        dfs.append(txn)
        
    except: 
        pass

dfs = pd.concat(dfs)

In [12]:
df.shape[0]
dfs.shape[0]
for day in [1, 2, 3]:
    dfs[f'event_{day}'].value_counts()

15470

9742

False    9121
True      621
Name: event_1, dtype: int64

False    8750
True      992
Name: event_2, dtype: int64

False    8587
True     1155
Name: event_3, dtype: int64

In [14]:
df.shape[0]
con = dfs.loc[dfs['amountUSD'] > 10000]
con.shape[0]
for day in [1, 2, 3]:
    con[f'event_{day}'].value_counts()

15470

630

False    509
True     121
Name: event_1, dtype: int64

False    459
True     171
Name: event_2, dtype: int64

False    434
True     196
Name: event_3, dtype: int64

# Layer One

In [17]:
df = pd.read_csv(f'{WD}connected_wallet_transaction/list_lp/panel_buy_ret.csv')
df['date'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%d')

for day in [1, 2, 3]:
    df[f'event_{day}'] = False

dfs = []
for df_idx, df_row in df.drop_duplicates(['insider_contractAddress', 'chain']).iterrows():
    try:
        contractAddress = df_row['insider_contractAddress']
        chain = df_row['chain']
        
        dft = pd.read_csv(f'{WD}tweet_all/csv/{contractAddress}_tweets.csv')
        dft['date'] = pd.to_datetime(dft['timestamp']).dt.strftime('%Y-%m-%d')

        start = dft['date'].min()
        end = dft['date'].max()

        txn = df.loc[(df['insider_contractAddress'] == contractAddress)&
                     (df['chain'] == chain)&
                     (pd.to_datetime(df['date']) > pd.to_datetime(start) + datetime.timedelta(days=3))&
                     (pd.to_datetime(df['date']) < pd.to_datetime(end) - datetime.timedelta(days=3))].copy()

        dft['text'] = dft['text'].apply(lambda x: x.lower())
        dft = dft.loc[[True if re.search('listing', i) else False for i in dft['text']]]
        events = dft['date'].unique()

        for event in events:
            for idx, row in txn.iterrows():
                index = row['id']
                date = row['date']
                for day in [1, 2, 3]:
                    if (pd.to_datetime(date) > pd.to_datetime(event) - datetime.timedelta(days=day))&(pd.to_datetime(date) < pd.to_datetime(event) + datetime.timedelta(days=day)):
                        txn.loc[(txn['id'] == index), f'event_{day}'] = True

        dfs.append(txn)
        
    except: 
        pass

dfs = pd.concat(dfs)

/tmp/ipykernel_655517/1758251824.py:1: DtypeWarning: Columns (4,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{WD}connected_wallet_transaction/list_lp/panel_buy_ret.csv')


In [18]:
df.shape[0]
dfs.shape[0]
for day in [1, 2, 3]:
    dfs[f'event_{day}'].value_counts()

17191

12610

False    11817
True       793
Name: event_1, dtype: int64

False    10805
True      1805
Name: event_2, dtype: int64

False    10126
True      2484
Name: event_3, dtype: int64

In [20]:
df.shape[0]
con = dfs.loc[dfs['amountUSD'] > 10000]
con.shape[0]
for day in [1, 2, 3]:
    con[f'event_{day}'].value_counts()

17191

920

False    846
True      74
Name: event_1, dtype: int64

False    789
True     131
Name: event_2, dtype: int64

False    729
True     191
Name: event_3, dtype: int64

# Test

In [16]:
df['insider_token_name'].nunique()
df['contractAddress'].nunique()

904

935

In [129]:
api = tweepy_config()

contractAddress = '0x24e89bdf2f65326b94e36978a7edeac63623dafa'
chain = 'ethereum'
tokenName = 'Tiger King'


name = official_twitter(contractAddress=contractAddress,
                        chain=chain,
                        tokenName = tokenName)
name
users = api.search_users(name)

dft = get_tweets(user_id=users[0].id)

dft

'Tiger_King_Coin'

,timestamp,text
0,2022-08-24 21:06:40+00:00,RT @SwissAssAssin91: Day 200 of tweeting @elonmusk until he helps us #SavetheTigers with the 👑 o...
1,2022-08-24 21:04:06+00:00,No #memecoin community can match the passion and dedication of @Tiger_King_Coin. Build in the Be...
2,2022-08-24 21:00:53+00:00,RT @SwissAssAssin91: 200 days of tweeting @elonmusk what other community has this type of passio...
3,2022-08-23 02:46:38+00:00,#SavetheTigers https://t.co/njgFwqmmlt
4,2022-08-23 02:45:48+00:00,RT @Tigers4Ever2010: In these extremely difficult times a like or share could put Tigers4Ever in...
...,...,...
1356,2021-05-11 04:06:30+00:00,$TKING is the Big Cat - King of the Jungle \nSoon to be king of the Memes we are looking to buil...
1357,2021-05-11 04:00:13+00:00,RT @cryptolego1: Almost 500k liquidity 🔒 5M volume in 4 hours!? 🤯 636 holders 💼 Grab your 🍿 and ...
1358,2021-05-11 03:56:20+00:00,RT @aloha_capital_: $TKING Tiger king Twitter is live. \n\nGo Follow 👇🏻👇🏻👇🏻 https://t.co/Ee4dM0aD4c
1359,2021-05-11 03:45:37+00:00,$TKING launched on UNISWAP already creating a stir amongst the biggest MEME coins and currently ...


In [147]:
dft['text'] = dft['text'].apply(lambda x: x.lower())
dft = dft.loc[[True if re.search('listing', i) else False for i in dft['text']]]
dft['timestamp'] = pd.to_datetime(dft['timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
events = dft['timestamp'].unique()

final = []
for event in events:
    txn = []
    for idx, row in df.loc[df['contractAddress'] == contractAddress].iterrows():
        index = row['id']
        if (pd.to_datetime(row['timestamp']) >= (pd.to_datetime(event) - datetime.timedelta(days=3)))&(pd.to_datetime(row['timestamp']) <= (pd.to_datetime(event) + datetime.timedelta(days=3))):
            txn.append(df.loc[df['id'] == index])

    if len(txn) != 0:
        txn = pd.concat(txn)
        txn['event'] = event
        txn['time_delta'] = pd.to_datetime(txn['timestamp']) - pd.to_datetime(txn['event'])
        final.append(txn)
        
final = pd.concat(final)
final

,Unnamed: 0,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_root,contractAddress,from,to,lp,buy,ret_1,ret_2,ret_3,ret_-1,ret_-2,ret_-3,insider_token_name,event,time_delta
30,30,719.771991,0x0df250704658224ca55ae95e82bae6e63b5aaefa4db47f5c2f137e3c36529057#708,211,0x83da32c690afc12b08621cdae37268eeda4a964c,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,5535243264563745790662704,-191389.0,2021-08-20 12:05:42,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0df250704658224ca55ae95e82bae6e63b5aaefa4db47f5c2f137e3c36529057,0.000000e+00,46000000.0,0.224189,0.000000,True,uniswapv3_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,-719.771991,-0.042418,-0.066189,-0.013041,0.139668,-0.070384,-0.345031,Tiger King,2021-08-17 21:21:39,2 days 14:44:03
38,38,7478.974558,0x63dcb02114a962bbb82a1fb50f1f8ea27aaf4637fcbce5f7219753ed8b580e12#83,48,0x83da32c690afc12b08621cdae37268eeda4a964c,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,5753880147597764424946029,-190614.0,2021-08-18 02:42:10,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x63dcb02114a962bbb82a1fb50f1f8ea27aaf4637fcbce5f7219753ed8b580e12,0.000000e+00,463000000.0,2.473233,0.000000,True,uniswapv3_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,-7478.974558,0.139668,0.146603,0.104186,-0.274647,NaN,NaN,Tiger King,2021-08-17 21:21:39,0 days 05:20:31
57,57,5565.306641,0xf5ad728326082b1d4befa92a5338c33284b1ffa1916f2b1a10fc3112a1d96cd9#241,6,0x83da32c690afc12b08621cdae37268eeda4a964c,0x83da32c690afc12b08621cdae37268eeda4a964c,0xe592427a0aece92de3edee1f18e0157c05861564,5435112145683137373079717,-191754.0,2021-08-18 12:05:35,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf5ad728326082b1d4befa92a5338c33284b1ffa1916f2b1a10fc3112a1d96cd9,3.968941e+08,0.0,0.000000,1.850000,True,uniswapv3_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,NaN,NaN,False,5565.306641,0.139668,0.146603,0.104186,-0.274647,NaN,NaN,Tiger King,2021-08-17 21:21:39,0 days 14:43:56
1114,1182,4707.722246,0x1abac089b4463e0fb0d10d24e720200f73bb7b7e3e2a8f8ab0d934a1fe8fcebe-0,8,NaN,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,NaN,NaN,2021-08-16 15:23:20,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1abac089b4463e0fb0d10d24e720200f73bb7b7e3e2a8f8ab0d934a1fe8fcebe,1.921182e+08,0.0,0.000000,1.471628,True,uniswapv2_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0x83da32c690afc12b08621cdae37268eeda4a964c,0x83da32c690afc12b08621cdae37268eeda4a964c,False,4707.722246,-0.274647,-0.484699,-0.345031,NaN,NaN,NaN,Tiger King,2021-08-17 21:21:39,-2 days +18:01:41
1115,1183,32399.939227,0x4394130f762c7ab53db85c7923d50f1e3c8a89119d5f925bc1f85c36dc1c39b3-0,8,NaN,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,NaN,NaN,2021-08-16 20:16:13,Tiger King,Wrapped Ether,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x4394130f762c7ab53db85c7923d50f1e3c8a89119d5f925bc1f85c36dc1c39b3,1.273797e+09,0.0,0.000000,10.300000,True,uniswapv2_ethereum,ethereum,0x83da32c690afc12b08621cdae37268eeda4a964c,0x24e89bdf2f65326b94e36978a7edeac63623dafa,0x83da32c690afc12b08621cdae37268eeda4a964c,0x83da32c690afc12b08621cdae37268eeda4a964c,False,32399.939227,-0.274647,-0.484699,-0.345031,NaN,NaN,NaN,Tiger King,2021-08-17 21:21:39,-2 days +22:54:34
1116,1184,600.619450,0x4c3b5bb809cf918f1cffd9b108fb5e5e9cef56ab

In [117]:
def tweepy_config():
    config = configparser.ConfigParser()
    config.read('config.ini')
    api_key = config['twitter']['api_key']
    api_key_secret = config['twitter']['api_key_secret']
    access_token = config['twitter']['access_token']
    access_token_secret = config['twitter']['access_token_secret']
    auth = tweepy.OAuthHandler(api_key, api_key_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api
    

In [158]:
def official_twitter(contractAddress, chain, tokenName):
    
    time.sleep(2 + random.random()*2)
    
    
    if chain == 'ethereum': url = f'https://etherscan.io/token/{contractAddress}'
    if chain == 'binance': url = f'https://bscscan.com/token/{contractAddress}'
    if chain == 'polygon': url = f'https://polygonscan.com/token/{contractAddress}'
    if chain == 'fantom': url = f'https://ftmscan.com/token/{contractAddress}'
    if chain == 'avalanche': url = f'https://snowtrace.io/token/{contractAddress}'
    
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
    r = requests.get(url=url, headers=headers)
    bs = BeautifulSoup(r.text)
    twitter_url = ''
    for social in bs.find_all('a', attrs={'class':'tooltips'}):
        if re.search('twitter', social['href']) != None: twitter_url = social['href']

    if twitter_url != '':
        twitter = re.findall('(?<=com/).*$', twitter_url)[0]
        return twitter

    else:
        return tokenName

In [128]:
def get_tweets(user_id):
    timestamp = []
    text = []
    iter_num = 0

    while True:
        iter_num += 1
        if iter_num == 1:
            tweets = api.user_timeline(user_id=user_id, 
                                   # 200 is the maximum allowed count
                                   count=200,
                                   exclude_replies = True,
                                   include_rts = True,
                                   # Necessary to keep full_text 
                                   # otherwise only the first 140 words are extracted
                                   tweet_mode = 'extended'
                               )
        else:
            tweets = api.user_timeline(user_id=user_id, 
                                       # 200 is the maximum allowed count
                                       count=200,
                                       exclude_replies = True,
                                       include_rts = True,
                                       max_id = oldest_id - 1,
                                       # Necessary to keep full_text 
                                       # otherwise only the first 140 words are extracted
                                       tweet_mode = 'extended'
                                   )
        if len(tweets) == 0: break

        for tweet in tweets:
            timestamp.append(tweet.created_at)
            text.append(tweet.full_text)

        oldest_id = tweets[-1].id

    dft = pd.DataFrame({'timestamp':timestamp, 'text':text})
    return dft